In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time
import pandas as pd
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
plt.style.use('ggplot')
%matplotlib inline

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./data/mnist', validation_size=0, one_hot=True)
test_len = mnist.test.images.shape[0]
validation_len = int(test_len * 0.1)
num_label=10
train_x = mnist.train.images
test_x = mnist.test.images[validation_len : test_len, :]
validation_x = mnist.test.images[ : validation_len, :]

train_y = mnist.train.labels
test_y = mnist.test.labels[validation_len : test_len]
validation_y = mnist.test.labels[ : validation_len]

print("\nTraining X shape: " + str(train_x.shape))
print("Testing X shape: " + str(test_x.shape))
print("Validation X shape: " + str(validation_x.shape))

print("\nTraining Y shape: " + str(train_y.shape))
print("Testing Y shape: " + str(test_y.shape))
print("Validation Y shape: " + str(validation_y.shape))

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz

Training X shape: (60000, 784)
Testing X shape: (9000, 784)
Validation X shape: (1000, 784)

Training Y shape: (60000, 10)
Testing Y shape: (9000, 10)
Validation Y shape: (1000, 10)


In [3]:
import scipy.io as sio
import scipy.misc as scimisc
safe_list = [0,2,5,6,8,12,13,14,15,16,17,18,19,21,26]  # other alphabets have characters which look like digits
m = sio.loadmat("./data/data_background.mat")

squished_set = []
for safe_number in safe_list:
    for alphabet in m['images'][safe_number]:
        for letters in alphabet:
            for letter in letters:
                for example in letter:
                    squished_set.append(scimisc.imresize(1 - example[0], (28,28)).reshape(1, 28*28))

safe_images = np.concatenate(squished_set, axis=0)

/home/senqi/anaconda3/lib/python3.5/site-packages/scipy/misc/pilutil.py:480: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/home/senqi/anaconda3/lib/python3.5/site-packages/scipy/misc/pilutil.py:483: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


In [7]:
M = 256
D=7*7*64
h1=512
D2=10
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"  
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

tf.reset_default_graph()
# variables
X = tf.placeholder(tf.float32, shape = [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None,10])

w_conv1 = tf.get_variable('w_conv1', [5,5, 1,32], initializer = tf.contrib.layers.xavier_initializer())
w_conv2 = tf.get_variable('w_conv2', [5,5,32,64], initializer = tf.contrib.layers.xavier_initializer())

w0 = tf.get_variable('w_fc1', [7*7*64, h1], initializer = tf.contrib.layers.xavier_initializer())
b0 = tf.get_variable('b_fc1', [1,h1], initializer = tf.contrib.layers.xavier_initializer())
w1 = tf.get_variable('w_fc2', [512, 10], initializer = tf.contrib.layers.xavier_initializer())
b1 = tf.get_variable('b_fc2', [1,10], initializer = tf.contrib.layers.xavier_initializer())

# network
b=True
con1 = tf.nn.conv2d(X, w_conv1, strides=[1, 1, 1, 1], padding='SAME')
h_conv1 = tf.nn.relu(tf.layers.batch_normalization(con1, training=b))
h_pool1 = max_pool_2x2(h_conv1)

con2 = tf.nn.conv2d(h_pool1, w_conv2, strides=[1, 1, 1, 1], padding='SAME')
h_conv2 = tf.nn.relu(tf.layers.batch_normalization(con2, training=b))
h_pool2 = max_pool_2x2(h_conv2)

h_pool2_flat = tf.layers.flatten(h_pool2)

h = tf.nn.relu(tf.matmul(h_pool2_flat, w0) + b0)
logits = tf.matmul(h, w1) + b1
output = tf.nn.softmax(logits)#batch*10

#evaluation
probs = tf.reduce_sum(output*Y,1)#batch*1
p=tf.reduce_mean(probs,0)#积分

max_p = tf.reduce_max(output,1)
ent = tf.reduce_sum(-tf.log(output+1e-11)*output,1)

correct_pred = tf.equal(tf.argmax(output,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# training
a = 0.00001
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_step = tf.train.AdamOptimizer(a).minimize(-p)
saver = tf.train.Saver()

with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    model_id=0
    #save_path = saver.restore(sess, "./baseline0/classification_mnist1/model.ckpt")
    for i in range(60000*100):
        batch = mnist.train.next_batch(M)
        x_batch = np.reshape(batch[0],[M, 28, 28, 1])
        y_batch = batch[1]
        
        if i%100==0:
            b=False
            maxp_OOD = []
            ent_OOD = []
            
            test_accuracy = []
            maxp_in = []
            ent_in = []
            #accu = sess.run(accuracy,{X: np.reshape(mnist.train.images,[55000,28,28,1]), Y: mnist.train.labels})
                        
            
            for j in range(0,100):
                imgs_in = mnist.test.images[j*100:(j+1)*100]
                labels_in = mnist.test.labels[j*100:(j+1)*100]
                acc = sess.run(accuracy,{X:np.reshape(imgs_in,[100,28,28,1]),Y:labels_in})
                maxp_in1 = sess.run(max_p,{X:np.reshape(imgs_in,[100,28,28,1])})
                ent_in1 = sess.run(ent,{X:np.reshape(imgs_in,[100,28,28,1])})
                test_accuracy.append(acc)
                maxp_in.extend(maxp_in1)
                ent_in.extend(ent_in1)
            test_accuracy = np.array(test_accuracy)
            maxp_in = np.array(maxp_in)
            ent_in = np.array(ent_in)
                
            for k in range(0,92):
                imgs_OOD = safe_images[k*100:(k+1)*100]
                maxp_OOD1 = sess.run(max_p,{X:np.reshape(imgs_OOD,[100,28,28,1])})
                ent_OOD1 = sess.run(ent,{X:np.reshape(imgs_OOD,[100,28,28,1])})
                maxp_OOD.extend(maxp_OOD1)
                ent_OOD.extend(ent_OOD1)
            maxp_OOD = np.array(maxp_OOD)
            ent_OOD = np.array(ent_OOD)

            validation_error = sess.run(accuracy, {X:np.reshape(validation_x,[1000,28,28,1]), Y:validation_y})
            test_error = np.mean(np.array(test_accuracy))
            '''
            test_error = sess.run(accuracy, {X:np.reshape(mnist.test.images,[10000,28,28,1]), Y:mnist.test.labels[:5000]})
            validation_error = sess.run(accuracy, {X:np.reshape(validation_x,[1000,28,28,1]), Y:validation_y})
            maxp_OOD = sess.run(max_p,{X:np.reshape(safe_images,[9200,28,28,1])})
            maxp_in = sess.run(max_p, {X:np.reshape(mnist.test.images,[10000,28,28,1])})
            ent_OOD = sess.run(ent,{X:np.reshape(safe_images,[9200,28,28,1])})
            ent_in = sess.run(ent, {X:np.reshape(mnist.test.images,[10000,28,28,1])})
            '''
            print ("time:",i, time.time() - start_time, "test accuracy", test_error, "validation_error",validation_error)
            
            print("maxp_OOD:",np.mean(maxp_OOD>0.99),np.mean(maxp_OOD),np.std(maxp_OOD))
            print("maxp_inD:",np.mean(maxp_in>0.99),np.mean(maxp_in),np.std(maxp_in))
            print("ent_OOD:",np.mean(ent_OOD),np.std(ent_OOD), "ent_in:", np.mean(ent_in),np.std(ent_in))
            
            safe, risky = np.reshape(maxp_in,[10000,1]), np.reshape(maxp_OOD,[9200,1])
            labels = np.zeros((safe.shape[0] + risky.shape[0]), dtype=np.int32)
            labels[:safe.shape[0]] += 1
            examples = np.squeeze(np.vstack((safe, risky)))
            print('AUPR_p:', round(100*average_precision_score(labels, examples), 2))
            print('AUROC_p:', round(100*roc_auc_score(labels, examples), 2))

            risky,safe = np.reshape(ent_in,[10000,1]), np.reshape(ent_OOD,[9200,1])
            labels = np.zeros((safe.shape[0] + risky.shape[0]), dtype=np.int32)
            labels[:safe.shape[0]] += 1
            examples = np.squeeze(np.vstack((safe, risky)))
            print('AUPR_entropy:', round(100*average_precision_score(labels, examples), 2))
            print('AUROC_entropy:', round(100*roc_auc_score(labels, examples), 2))
            print("############################################")
            
        if i < 1000:
            a = 0.001
            b = True
            sess.run(train_step,{X: x_batch, Y: y_batch})
        elif i < 5000:
            a = 0.0005
            b = True
            sess.run(train_step,{X: x_batch, Y: y_batch})
        elif i < 10000:
            a = 0.0001
            b = True
            sess.run(train_step,{X: x_batch, Y: y_batch})
        else:
            a = 0.00001
            b = True
            sess.run(train_step,{X: x_batch, Y: y_batch})
        
        if i%500==0:
            save_path = saver.save(sess, "./baseline0/classification_mnist%s/model.ckpt" % model_id)
            model_id+=1

KeyboardInterrupt: 

In [6]:
1+1

2